In [2]:
import pandas as pd
import numpy as np
import vnstock as vn

In [3]:
# Load data
START = "2019-01-01"
END = "2024-02-01"
def load_data(ticker):
        data = vn.stock_historical_data(ticker, START, END)    
        return data
data = load_data('VCB') 
data = data.set_index('time')
data

,open,high,low,close,volume,ticker
time,,,,,,
2019-01-02,34750,35010,34490,34560,1081640,VCB
2019-01-03,34820,34880,33980,34560,1071350,VCB
2019-01-04,34370,35080,33910,35080,1307310,VCB
2019-01-07,35660,35780,35140,35200,1175810,VCB
2019-01-08,35460,35590,35140,35460,1318810,VCB
...,...,...,...,...,...,...
2024-01-26,91600,92000,91400,92000,953100,VCB
2024-01-29,91800,92000,90900,91200,730500,VCB
2024-01-30,91200,91200,90100,91000,888600,VCB


# Volume

In [4]:
mean_20 = data['volume'].rolling(window=20).mean() #Tính trung bình 20 phiên gần nhất
data['on-balance_volume'] = data['volume']/mean_20 #Tính on-balance volume
data

,open,high,low,close,volume,ticker,on-balance_volume
time,,,,,,,
2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN
2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN
2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN
2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN
2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN
...,...,...,...,...,...,...,...
2024-01-26,91600,92000,91400,92000,953100,VCB,0.643849
2024-01-29,91800,92000,90900,91200,730500,VCB,0.508792
2024-01-30,91200,91200,90100,91000,888600,VCB,0.638869


In [5]:
def label_values(a):
    if pd.isnull(a):
        return a
    elif a < 0.75:
        return 'low'
    elif 0.75 <= a <= 1.25:
        return 'medium'
    else:
        return 'high'
        
data['OBV_label'] = data['on-balance_volume'].apply(label_values)
data.sample(10)

,open,high,low,close,volume,ticker,on-balance_volume,OBV_label
time,,,,,,,,
2020-09-17,53670,53990,53340,53670,1043250,VCB,0.782205,medium
2022-12-12,66720,68160,65540,65540,2105700,VCB,1.364869,high
2019-06-06,43010,43010,41910,42490,394550,VCB,0.765841,medium
2019-06-25,46940,46940,46420,46680,390840,VCB,0.526415,low
2020-11-20,59200,60170,58610,60170,1231960,VCB,1.400032,high
2019-01-10,35980,35980,35460,35460,794870,VCB,NaN,NaN
2022-03-04,72140,72400,71800,71890,1058100,VCB,0.686527,low
2019-09-23,52610,52810,50940,50940,1187100,VCB,1.579004,high
2022-01-19,74090,74090,71970,73500,914800,VCB,0.702401,low


# Bar

In [6]:
def compare_close_prices(t, t_minus_1):
    if t > t_minus_1:
        return 'up-bar'  
    else:
        return 'down-bar'

data['close_t_minus_1'] = data['close'].shift(1) #Giá đóng cửa ngày t-1

data['bar_type'] = data.apply(lambda row: compare_close_prices(row['close'], row['close_t_minus_1']), axis=1)
data.sample(10)

,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type
time,,,,,,,,,,
2023-06-27,84670,85520,84420,84670,685900,VCB,0.757340,medium,84670.0,down-bar
2021-05-06,66370,66700,64930,65120,1161200,VCB,0.557040,low,66370.0,down-bar
2024-01-31,90600,90600,88500,88500,945600,VCB,0.690458,low,91000.0,down-bar
2022-08-30,69860,72820,69860,72820,2502800,VCB,2.179798,high,69860.0,up-bar
2020-01-15,58290,58290,57770,58090,1471860,VCB,1.959217,high,58160.0,down-bar
2023-05-26,78920,79170,77900,77900,283700,VCB,0.501246,low,78920.0,down-bar
2020-06-22,55290,55680,55290,55620,1308600,VCB,1.347742,high,55550.0,up-bar
2021-03-02,65580,66760,64269,65120,1189600,VCB,0.835892,medium,65060.0,up-bar
2023-07-03,84670,85350,84500,84670,490100,VCB,0.558204,low,84670.0,down-bar


# Spread

In [7]:
data['para'] =abs(data['close'] - data['open'])/(data['high'] - data['low'])
data

,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para
time,,,,,,,,,,,
2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN,NaN,NaN,down-bar,0.365385
2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN,NaN,34560.0,down-bar,0.288889
2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN,NaN,34560.0,up-bar,0.606838
2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN,NaN,35080.0,up-bar,0.718750
2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN,NaN,35200.0,up-bar,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
2024-01-26,91600,92000,91400,92000,953100,VCB,0.643849,low,91000.0,up-bar,0.666667
2024-01-29,91800,92000,90900,91200,730500,VCB,0.508792,low,92000.0,down-bar,0.545455
2024-01-30,91200,91200,90100,91000,888600,VCB,0.638869,low,91200.0,down-bar,0.181818


In [8]:
def label_spread(a):
    if 0 < a < 0.3: return 'low'
    elif 0.3 <= a <= 0.7: return 'medium'
    else: return 'high'

data['labe_spread'] = data['para'].apply(label_spread)
data

,open,high,low,close,volume,ticker,on-balance_volume,OBV_label,close_t_minus_1,bar_type,para,labe_spread
time,,,,,,,,,,,,
2019-01-02,34750,35010,34490,34560,1081640,VCB,NaN,NaN,NaN,down-bar,0.365385,medium
2019-01-03,34820,34880,33980,34560,1071350,VCB,NaN,NaN,34560.0,down-bar,0.288889,low
2019-01-04,34370,35080,33910,35080,1307310,VCB,NaN,NaN,34560.0,up-bar,0.606838,medium
2019-01-07,35660,35780,35140,35200,1175810,VCB,NaN,NaN,35080.0,up-bar,0.718750,high
2019-01-08,35460,35590,35140,35460,1318810,VCB,NaN,NaN,35200.0,up-bar,0.000000,high
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-26,91600,92000,91400,92000,953100,VCB,0.643849,low,91000.0,up-bar,0.666667,medium
2024-01-29,91800,92000,90900,91200,730500,VCB,0.508792,low,92000.0,down-bar,0.545455,medium
2024-01-30,91200,91200,90100,91000,888600,VCB,0.638869,low,91200.0,down-bar,0.181818,low
